# Exercise 7. Hyperparameter Tuning

### Hyperparameter Tuning
 Now that you have trained an initial model, you can tune the hyperparameters of this model to optimize model performance. Azure ML allows you to automate this tuning, in an efficient manner via early termination of poorly performing runs.

You can configure your Hyperparamter Tuning experiment by specifying the following info -
- Define the hyparparameter space - specify ranges, distribution and sampling
- Early Termination policy
- Optimization metric
- Resource / Compute budget
- Desired concurrency


In [ ]:
from azureml.train.dnn import PyTorch

script_params = {
    '--data_dir': ds_data.as_mount(),
    '--num_epochs': 40,
    '--output_dir': './outputs',
    '--log_dir': './logs',
    '--mode': 'fine_tune'
}

estimator10_40 = PyTorch(source_directory='.', 
                    script_params=script_params,
                    compute_target=compute_target, 
                    entry_script='pytorch_train.py',
                    pip_packages=['tensorboardX'],
                    use_gpu=True)

In [ ]:
from azureml.train.hyperdrive import *

ps = RandomParameterSampling(
    {
        '--momentum': uniform(0.6,0.99),
        '--learning_rate': loguniform(-6, -3)
    }
)

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2)


hdc = HyperDriveRunConfig(estimator=estimator10_40, 
                          hyperparameter_sampling=ps, 
                          policy=policy, 
                          primary_metric_name='best_val_acc', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=50,
                          max_concurrent_runs=4)

hd_run = experiment.submit(hdc)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(hd_run).show()

While the jobs is running, take a look at the [Hyperdrive documentation](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive?view=azure-ml-py) to see what other options Hyperdrive offers.

In [ ]:
# at any time, you can pull out the metrics generated so far from the runs
hd_run.get_metrics()